將每個Tool最後一次RUL=0之後的資料都刪掉

In [1]:
# Step 1: 匯入套件
import pandas as pd

# Step 2: 讀取 CSV
df = pd.read_csv("./csv/dataset.csv", parse_dates=["DateTime"])

print("原始資料筆數:", len(df))
df.head()


原始資料筆數: 356


,DateTime,Tool,FPD,Temperature,RUL
0,2022-01-01,A,0.62,23.11,0.0
1,2022-01-08,A,0.87,23.47,0.0
2,2022-02-05,A,0.34,22.19,77.0
3,2022-02-26,A,0.48,22.69,73.0
4,2022-03-16,A,0.62,23.76,0.0


In [2]:
# Step 3: 定義函數
def remove_after_last_failure(data: pd.DataFrame) -> pd.DataFrame:
    """
    移除每個 Tool 中，最後一次 RUL=0 之後的所有資料
    """
    cleaned_list = []
    
    for tool, group in data.groupby("Tool"):
        # 找出最後一次 RUL=0 的位置
        last_failure_idx = group[group["RUL"] == 0].index.max()
        
        if pd.isna(last_failure_idx):
            # 如果這個 Tool 完全沒有 RUL=0，就保留全部
            cleaned_group = group
        else:
            # 只保留最後一次失效 (RUL=0) 之前與當下的資料
            cleaned_group = group.loc[:last_failure_idx]
        
        cleaned_list.append(cleaned_group)
    
    # 合併
    cleaned_df = pd.concat(cleaned_list, ignore_index=True)
    return cleaned_df


In [6]:
# Step 4: 使用函數
df_cleaned = remove_after_last_failure(df)

print("清理後資料筆數:", len(df_cleaned))
df_cleaned.groupby("Tool")["RUL"].min()

# Step 5: (選擇性) 存回新的 CSV
df_cleaned.to_csv("./csv/dataset.csv", index=False)


清理後資料筆數: 353


利用時間序列插值 (time)進行差值

In [20]:
# Cell 1: 匯入與設定（不輸出）
import pandas as pd
import numpy as np

# I/O
SRC_CSV = "./csv/dataset.csv"  # 原始資料
OUT_CSV = "./csv/interpolated_dataset.csv"  # 輸出檔名

# 欄位與頻率
INTERP_COLS = ["FPD", "Temperature"]   # 要插值的連續型欄位
RESAMPLE_FREQ = "D"                     # 轉成每日

# 以插值後的 FPD 判定失效的門檻
FPD_FAILURE_THRESHOLD = 0.6

In [ ]:
# Cell 2: 讀檔與新增 Date 欄位（不輸出）
df = pd.read_csv(SRC_CSV, parse_dates=["DateTime"])
df.sort_values(["Tool", "DateTime"], inplace=True, ignore_index=True)

# 新增只含日期的欄位（物件型別為 datetime.date）
df["Date"] = pd.to_datetime(df["DateTime"]).dt.date

# 檢查結果
print(df)

      DateTime Tool   FPD  Temperature   RUL        Date
0   2022-01-01    A  0.62        23.11   0.0  2022-01-01
1   2022-01-08    A  0.87        23.47   0.0  2022-01-08
2   2022-02-05    A  0.34        22.19  77.0  2022-02-05
3   2022-02-26    A  0.48        22.69  73.0  2022-02-26
4   2022-03-16    A  0.62        23.76   0.0  2022-03-16
..         ...  ...   ...          ...   ...         ...
348 2024-10-25    E  0.38        23.40  79.0  2024-10-25
349 2024-11-22    E  0.45        23.12  49.0  2024-11-22
350 2024-12-02    E  0.89        22.71   0.0  2024-12-02
351 2024-12-17    E  0.71        23.44   0.0  2024-12-17
352 2024-12-28    E  0.85        23.68   0.0  2024-12-28

[353 rows x 6 columns]


In [21]:
# Cell 3: 單一 Tool -> 每日 resample + 時間插值（不輸出）
def resample_and_interpolate_single_tool(group: pd.DataFrame,
                                         interp_cols=INTERP_COLS,
                                         resample_freq=RESAMPLE_FREQ) -> pd.DataFrame:
    """
    對單一 Tool：
      1) 依 resample_freq 建立等間距時間格點
      2) 對指定欄位進行 method='time' 的插值
      3) 保留 Tool 與原始 RUL（作為參考；最終 RUL 會重算）
    回傳 index=DateTime 的 DataFrame
    """
    g = group.set_index("DateTime").sort_index()

    # 產生每日格點
    g_daily = g.asfreq(resample_freq)

    # 保留 Tool 標籤
    g_daily["Tool"] = g["Tool"].iloc[0]

    # 保留原始 RUL 供參考（非原始日期為 NaN）
    g_daily["RUL_orig"] = g["RUL"]

    # 對連續欄位做時間型插值
    for col in interp_cols:
        if col in g_daily.columns:
            g_daily[col] = g_daily[col].interpolate(method="time")

    return g_daily


In [22]:
# Cell 4: 依「插值後的 FPD 門檻」重新計算 RUL（不輸出）
def recompute_rul_from_interpolated_fpd(group_daily: pd.DataFrame,
                                        fpd_threshold: float = FPD_FAILURE_THRESHOLD) -> pd.DataFrame:
    """
    對單一 Tool 的每日資料，使用「插值後的 FPD > threshold」作為失效條件：
      - 當日 FPD > 門檻 -> RUL = 0
      - 往「過去」每天 +1，直到遇到下一個失效日再重置
    注意：插值後的 FPD 可能在原本沒有失效的日期也超過門檻，故 RUL=0 的天數可能變多。
    """
    g = group_daily.sort_index().copy()

    # 以插值後的 FPD 判定失效錨點
    failure_anchor = (g["FPD"] > fpd_threshold).fillna(False).to_numpy()

    # 由末日往前遞增計數
    rul = np.zeros(len(g), dtype=int)
    counter = 0
    for i in range(len(g) - 1, -1, -1):
        if failure_anchor[i]:
            counter = 0
            rul[i] = 0
        else:
            counter += 1
            rul[i] = counter

    g["RUL"] = rul
    return g


In [24]:
# Cell 5: 全流程：resample+interpolate → 依插值後 FPD 重算 RUL → 產出（不輸出）
# 1) 對每個 Tool 先做每日化與插值
daily = (
    df.groupby("Tool", group_keys=False)
      .apply(lambda g: resample_and_interpolate_single_tool(g))
)

# 2) 依插值後 FPD 門檻重新計算 RUL（RUL=0 可能變多）
daily = (
    daily.groupby("Tool", group_keys=False)
         .apply(lambda g: recompute_rul_from_interpolated_fpd(g, FPD_FAILURE_THRESHOLD))
)

# 3) 產生 Date 欄位（純日期）並整理欄位順序
daily = daily.reset_index()  # 還原 DateTime
daily["Date"] = pd.to_datetime(daily["DateTime"]).dt.date

# 若不想保留 RUL_orig，可取消註解下一行：
# daily = daily.drop(columns=["RUL_orig"])

# 最終欄位（可依需求調整順序）
cols = ["DateTime", "Date", "Tool"] + INTERP_COLS + ["RUL"]
daily = daily[[c for c in cols if c in daily.columns]]

# 4) 輸出 CSV
daily.to_csv(OUT_CSV, index=False)


/var/folders/2d/q366jpls0tn7mdr8lg1zq2k40000gn/T/ipykernel_65788/3315875139.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: resample_and_interpolate_single_tool(g))
/var/folders/2d/q366jpls0tn7mdr8lg1zq2k40000gn/T/ipykernel_65788/3315875139.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: recompute_rul_from_interpolated_fpd(g, FPD_FAILURE_THRESHOLD))


In [ ]:
# Cell 6: （可選）若要加入遲滯/緩衝（避免插值造成邊界抖動），可用下列範例函式替換判定（不輸出）
def failure_mask_with_hysteresis(fpd_series: pd.Series,
                                 on_threshold: float = 0.6,
                                 off_threshold: float = 0.58) -> np.ndarray:
    """
    範例：加入簡單遲滯（hysteresis），減少 FPD 僅略高/略低門檻時造成的頻繁切換。
    - 當前狀態為「失效」後，需低於 off_threshold 才解除；反之高於 on_threshold 才觸發。
    回傳布林陣列表示失效日。
    """
    fpd = fpd_series.to_numpy()
    fail = np.zeros_like(fpd, dtype=bool)
    state = False
    for i, x in enumerate(fpd):
        if not state:
            # 正常 -> 只有超過 on_threshold 才進入失效
            if x is not None and pd.notna(x) and x > on_threshold:
                state = True
        else:
            # 失效中 -> 只有低於 off_threshold 才解除
            if x is not None and pd.notna(x) and x < off_threshold:
                state = False
        fail[i] = state
    return fail

# 使用示例（把 Cell 4 的 failure_anchor 改為下列兩行）：
# failure_anchor = failure_mask_with_hysteresis(g["FPD"], on_threshold=0.6, off_threshold=0.58)
# （其餘流程不變）


       DateTime Tool       FPD  Temperature  RUL  RUL_orig
0    2022-01-01    A  0.620000    23.110000    0       0.0
1    2022-01-02    A  0.655714    23.161429    6       NaN
2    2022-01-03    A  0.691429    23.212857    5       NaN
3    2022-01-04    A  0.727143    23.264286    4       NaN
4    2022-01-05    A  0.762857    23.315714    3       NaN
...         ...  ...       ...          ...  ...       ...
5423 2024-12-24    E  0.799091    23.592727    4       NaN
5424 2024-12-25    E  0.811818    23.614545    3       NaN
5425 2024-12-26    E  0.824545    23.636364    2       NaN
5426 2024-12-27    E  0.837273    23.658182    1       NaN
5427 2024-12-28    E  0.850000    23.680000    0       0.0

[5428 rows x 6 columns]
